In [ ]:
! [ -e /content ] && pip install -Uqq fastai
! pip install dmba

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('review.csv', delimiter='\t')

In [ ]:
data=data.rename(columns={0: "Name", 2: "Date",1:"Rating",3:"Review"})

In [ ]:
#check data
data.head()


In [ ]:
#replace rating with 0 for negative, 1 for neutral, 2 for positive
data['Sentiment']=data['RatingValue'].replace({1:0,2:0,3:1,4:2,5:2})

In [ ]:
#Check data
data=data[['Review','Sentiment']]
data.head()

In [ ]:
#shuffle data and split
np.random.seed(seed=1)
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
valid = data[~msk]
train.to_csv('training.csv')
valid.to_csv('valid.csv')

In [ ]:
import fastai
print(fastai.__version__)
import fastai.text.all as ft
import pickle
import time 

In [ ]:
#load
with open('finetuned_lm_vocab.pkl', 'rb') as f:
  lm_vocab = pickle.load(f)

In [ ]:
# dataloader for the classifier
dls_cls = ft.TextDataLoaders.from_df(data, path='.', text_vocab=lm_vocab, valid_pct=0.2, is_lm=False, valid='test', text_col ='Review')
# dataloader for the language model
dls_lm = ft.TextDataLoaders.from_df(data, path='.', text_vocab=lm_vocab, valid_pct=0.2, is_lm=True,  valid='test', text_col ='Review')

In [ ]:
dls_cls.show_batch()

In [ ]:
dls_lm.show_batch()

In [ ]:
learn_lm = ft.language_model_learner(dls_lm, ft.AWD_LSTM, metrics=[ft.accuracy, ft.Perplexity()], wd=0.1).to_fp16()

In [ ]:
learn_lm.fit_one_cycle(1, 1e-2)


In [ ]:
learn_lm.save_encoder('/content/1epoch_lm')

In [ ]:
learn_lm.show_results()

In [ ]:

learn_lm.export(fname="1_tuned.pkl")

In [ ]:
learn_cls = ft.text_classifier_learner(dls_cls, ft.AWD_LSTM, drop_mult=0.5, metrics=ft.accuracy)

In [ ]:
learn_lm.model

In [ ]:
learn_cls.model

In [ ]:
learn_cls.fit_one_cycle(1, 2e-2)

In [ ]:
learn_cls.freeze_to(-3)
learn_cls.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn_cls.unfreeze()
learn_cls.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn_lm.model

In [ ]:
learn_cls.model

In [ ]:
learn_cls.export(fname="fine_tuned1.pkl")

In [ ]:
learn_cls.save_encoder('/content/fine_tuned_cls')

In [ ]:
learn_cls

In [ ]:
#test
learn_cls.predict('i like it')[0]


In [ ]:
valid.head()

In [ ]:
y_predict=[]
for review in valid['Review']:
  y_predict.append(learn_cls.predict(review))

In [ ]:
y_predict = np.array(y_predict)
y_predict = y_predict[:,0]

In [ ]:
y_true = valid['Sentiment']

In [ ]:
pip install dmba

In [ ]:
y_true

In [ ]:
y_predict = y_predict.astype(int)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from dmba import classificationSummary

print ('Accuracy:', accuracy_score(y_true, y_predict))
print ('F1 score:', f1_score(y_true, y_predict,average='weighted'))
classificationSummary(y_true, y_predict)

In [ ]:
learn_cls.summary()